In [ ]:
from IPython.display import HTML
from matplotlib_inline.backend_inline import set_matplotlib_formats

# Use inline backend so CSS scaling works on PNG/SVG outputs
set_matplotlib_formats('png')

HTML("""
<style>
/* =======================
   Matplotlib figure sizing
   ======================= */

/* Images (PNG/retina) */
.jp-RenderedImage img,
.jp-OutputArea-output img {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
  display: block;
  box-sizing: border-box;
}

/* SVG outputs */
.jp-RenderedSVG svg,
.jp-OutputArea-output svg {
  width: 100% !important;
  height: auto !important;
}

/* ipympl / canvas-backed figures (in case %matplotlib widget is used) */
.jupyter-matplotlib canvas.mpl-canvas,
.jp-OutputArea-output canvas.mpl-canvas,
.jp-OutputArea-output canvas {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
}

/* Reset container widths */
.jp-OutputArea-output .matplotlib-figure,
.jp-RenderedHTMLCommon .matplotlib-figure,
.jp-OutputArea-output .output_png {
  width: 100% !important;
}

/* =======================
   ipywidgets styling
   ======================= */

/* Make slider labels bold */
.widget-label {
    font-weight: bold;
}

/* Style ipywidgets buttons */
.jp-OutputArea .widget-button,
button.widget-button {
    font-weight: bold;
    background-color: #1976d2;  /* blue background */
    color: white;               /* white text */
    border: none;
    border-radius: 6px;
    padding: 6px 12px;
    font-size: 14px;
    cursor: pointer;
}

/* Button hover effect */
.jp-OutputArea .widget-button:hover,
button.widget-button:hover {
    background-color: #1259a4;  /* darker blue */
}
</style>
""")

# Signals in Sync ⏱️⏱️

## 🎯 Learning Goals

By the end of this notebook, you will be able to:  
- **Interpret** wave amplitude, frequency, and phase using **phasors**  
- **Analyze** the combination of waves and **explain** constructive versus destructive interference  
- **Describe and apply** the Nyquist rule by **relating** sampling rate to aliasing  


## **Adding Waves Together** 🌊➕🌊

One of the most powerful tools in engineering is the ability to **represent any physical signal as a combination of simple waves**. The key idea is that even the most complex signals can be **constructed** by adding together waves of different **amplitudes** and **frequencies**. 🌊🌀

### **Waves in Radar Systems**

In radar, waves are especially important because the **receiver records both the amplitude and phase** of the reflected electromagnetic pulses. 📡 

- The **transmitter and antenna** generate these pulses by making **electrons oscillate at a specific frequency**. ⚡
- When the transmitted wave encounters an object, part of it **reflects back** toward the radar. 🪞 
- This returning wave, in turn, **causes electrons in the receiver to oscillate**. 〰
- The receiver measures the properties of this oscillation, which are **the magnitude and phase of the reflected wave**. 📶

### **Phasors: A Visual Representation of Waves** 🔄➡️

Because waves follow a **periodic pattern**, we can visualize them as a **rotating arrow** 🔄. This concept is incredibly useful for **calculations and signal processing**.

This rotating arrow is called a **phasor**:
- The **length of the arrow** represents the **amplitude** of the wave. 📏 
- The **direction** of the arrow represents the **phase** of the wave. ∠

Remember that the **phase of a wave can be described by an angle**—it tells us **where the wave is in its cycle** at any given moment. In the simulation below, you can play with both the **frequency** and the **amplitude** of a sine wave, and see how these changes are represented by its corresponding **phasor**.

In [ ]:
def build_phasor_sim():
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.patches import FancyArrowPatch
    from ipywidgets import FloatSlider, Play, IntSlider, jslink, HBox, VBox, Layout, Output, Label, Button
    from IPython.display import display, clear_output

    # -----------------------
    # Fixed parameters
    # -----------------------
    DEFAULT_AMPLITUDE = 1.0
    DEFAULT_FREQUENCY = 1.0  # Hz
    DURATION = 2.0           # seconds shown on the sinusoid axis
    N_SAMPLES = 400          # points for the sinusoid curve
    N_FRAMES = 400           # animation frames
    DT = DURATION / N_FRAMES # seconds per frame

    # -----------------------
    # Widgets
    # -----------------------
    amplitude = FloatSlider(value=DEFAULT_AMPLITUDE, min=0.1, max=2.0, step=0.1, description="Amplitude A")
    amplitude.style.description_width = 'initial'
    frequency = FloatSlider(value=DEFAULT_FREQUENCY, min=0.1, max=5.0, step=0.1, description="Frequency f (Hz)")
    frequency.style.description_width = 'initial'
    play = Play(value=0, min=0, max=N_FRAMES-1, step=1, interval=40)
    t_slider = IntSlider(value=0, min=0, max=N_FRAMES-1, step=1, description="Frame")
    jslink((play, 'value'), (t_slider, 'value'))
    reset_btn = Button(description="Reset", button_style='info')
    save_btn = Button(description="Save Figure", button_style='success')
    readout = Label()
    controls = HBox([amplitude, frequency, play, t_slider, reset_btn, save_btn],
                    layout=Layout(align_items='center', column_gap='12px'))

    # -----------------------
    # Figure / Output
    # -----------------------
    out = Output()
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    plt.close(fig)  # prevent Voila/Jupyter from capturing a static snapshot

    # Static x-axis for the sinusoid
    t_axis = np.linspace(0, DURATION, N_SAMPLES)

    def _draw(frame_idx: int):
        # Current parameters & state
        A = float(amplitude.value)
        f = float(frequency.value)
        omega = 2 * np.pi * f
        t_now = frame_idx * DT
        angle = omega * t_now

        # Sinusoid for current params
        s = A * np.sin(omega * t_axis)
        y_now = A * np.sin(angle)

        with out:
            # Clear and redraw both subplots
            ax1.cla()
            ax2.cla()

            # --- Phasor subplot (I-Q plane) ---
            ax1.set_aspect('equal')
            pad = 1.2 * A
            ax1.set_xlim(-pad, pad)
            ax1.set_ylim(-pad, pad)
            ax1.set_title("Phasor Representation")
            ax1.set_xlabel("I (real)")
            ax1.set_ylabel("Q (imag)")
            ax1.grid(True, alpha=0.4)

            # Circle guide
            circle = plt.Circle((0, 0), A, fill=False, linestyle='--', alpha=0.5)
            ax1.add_artist(circle)

            # Phasor arrow
            x_end = A * np.cos(angle)
            y_end = A * np.sin(angle)
            arrow = FancyArrowPatch((0, 0), (x_end, y_end), mutation_scale=15, lw=2)
            ax1.add_patch(arrow)

            # --- Sinusoid subplot ---
            ax2.set_xlim(0, DURATION)
            ax2.set_ylim(-1.2*A, 1.2*A)
            ax2.set_title("Sinusoidal Wave")
            ax2.set_xlabel("Time (s)")
            ax2.set_ylabel("Amplitude")
            ax2.grid(True, alpha=0.4)

            # Plot the wave and the moving projection
            ax2.plot(t_axis, s, linestyle='--', lw=1.5, label="Sine wave")
            ax2.plot([t_now], [y_now], 'o', label="Wave measurement")
            ax2.plot([t_now, t_now], [-1.2*A, 1.2*A], ':', lw=1)
            ax2.legend(loc='best')

            # Readout
            readout.value = f"A = {A:.3g},  f = {f:.3g} Hz,  ω = {omega:.3g} rad/s,  t = {t_now:.2f} s"

            # Render
            clear_output(wait=True)
            display(fig)

    # Handlers
    def _reset(_):
        amplitude.value = DEFAULT_AMPLITUDE
        frequency.value = DEFAULT_FREQUENCY
        t_slider.value = 0
        play.value = 0
        _draw(0)

    def _save(_):
        import os
        os.makedirs('outputs', exist_ok=True)
        fname = f"outputs/phasor_A{float(amplitude.value):.1f}_f{float(frequency.value):.1f}_frame{int(t_slider.value)}.png"
        fig.savefig(fname, dpi=150, bbox_inches='tight')
        print(f"Saved {fname}")

    reset_btn.on_click(_reset)
    save_btn.on_click(_save)

    # Wire up
    amplitude.observe(lambda _: _draw(t_slider.value), names='value')
    frequency.observe(lambda _: _draw(t_slider.value), names='value')
    t_slider.observe(lambda ch: _draw(ch['new']), names='value')

    # Initial render
    _draw(t_slider.value)

    # Return a self-contained UI
    return VBox([controls, readout, out])

# Build and show (safe alongside other sims)
phasor_app = build_phasor_sim()
display(phasor_app)


_Figure: The phasor’s length equals amplitude and its angle maps to the point on the sine wave at the same time._

### 🧪 Try this
- Increase the **amplitude** and watch the phasor and sine scale together.
- Increase **frequency** to see faster rotation and quicker oscillations.
- Pause playback and scrub the frame slider to link **phase angle and time**.


### **Phasors in Radar Signal Processing** 📡⬆️

In **radar signal processing**, our goal is to **manipulate received waves** so that we can **add them together** in meaningful ways. Phasors provide a very convenient way to do that ↗️➕↘️.

Mathematically, phasor arrows are represented by **complex numbers**. Despite the name, there’s nothing truly “complex” about them—they're simply a convenient way to describe arrows with **both magnitude and direction**.

> In fact, the renowned German mathematician **Carl Friedrich Gauss**, one of the pioneers of complex numbers, disliked the term “complex” because he found it misleading. Why would anybody call the **vertical component** of the arrow an "imaginary" number?

> **💡 Side Note: I and Q – The Components of a Phasor**  
> In electrical engineering, you'll often encounter the terms **I (in-phase)** and **Q (quadrature)**.  
> These represent the **horizontal** and **vertical** components of a phasor, respectively:  
> - **I** = *in-phase* = horizontal axis, also called the real part.  
> - **Q** = *quadrature* = vertical axis, also called the imaginary part.  
>  
> This is just another way to express the same information as **amplitude and phase**, and it's often more convenient in signal processing and radar systems.

## **Why Is Phase So Important?** 📡

A **radar system** transmits **precisely controlled electromagnetic waves**, carefully defining their **amplitude** 📶 and **phase** 🔄. When these waves reflect off objects and return to the receiver, the system measures both **the amplitude and phase** of the received signal with high accuracy.  

Crucially, the **phase** 🔄 of the returning wave carries valuable information, as it reveals **how far the wave has traveled**. Consider the **typical wavelength for microwave radars**, which is about **3 cm**. By analyzing the **phase** of the received signals, we can detect changes in distance that are **a fraction of the wavelength**, in the order of **millimeters**. 📏✨  

> **This means we can measure millimeter-level changes in distance from hundreds of kilometers away using microwaves.** 🤯📡  

---

## **Wave Superposition: How Waves Combine** 🌊➕🌊

Picture a **very long rope** 🪢 with you at one end and a friend far away at the other. You both start **wiggling** your ends 🎯 to send waves down the rope. As the waves travel **toward each other** ↔️, they eventually **overlap** in the middle. In that overlap region, the rope doesn’t choose one wave or the other—the waves **are superimposed, that is added together** ➕. This is the **principle of superposition** ⚡.

Let's think about the different scenarios:
- **In phase (crests meet crests)** ⛰️➕⛰️: The displacements point the same way, so they **add**. The rope rises higher than either wave alone → **constructive interference** 🚀 (stronger wave).  
- **Out of phase (crest meets trough)** ⛰️➖⛰️: The displacements point opposite ways, so they **subtract**. With equal amplitudes, they **cancel completely** ❌ → **destructive interference** 🛑 (flat rope at that point).  
- **Anything in between** 🔄: Most of the time the phase difference isn’t exactly 0° or 180°, so the result is **partial reinforcement or partial cancellation**—somewhere between bigger and smaller.

💡 **Key idea:** The **phase difference** 🎚️ (how “lined up” the ripples are) decides the outcome.  
Line them up → louder 📢.  
Oppose them → quieter 🤫.  
Mix of both → something in between ⚖️.

The simulation below brings the rope example to life: two waves 🌊 travel toward each other along the rope at a constant speed 🚀 (just like radio waves, which always move at the speed of light). As they collide in the middle, the rope seems to ripple and dance ✨ — showing how **interference creates new wave patterns**.  

🎚️ **What you can adjust:**  
- **Amplitude** of each packet (left ↔️ right)  
- **Wavelength** (distance between ripples)  
- **Packet width** (how broad or narrow each wave packet is)  

👀 **What to observe:**  
- Equal amplitudes → strong **constructive interference** when crests overlap.  
- Unequal amplitudes → the larger wave dominates while the smaller one gently modulates it.  
- Narrow vs. wide packets → overlap becomes a quick flash ⚡ or a long, drawn-out interaction.  
- Changing the wavelength → ripples inside the packets stretch apart or squeeze closer together.  

In [ ]:
def build_rope_superposition_sim():
    """
    Voila-ready rope superposition demo:
    Two short sinusoidal wave packets travel toward each other,
    interfere in the middle, and pass through.

    Returns
    -------
    ipywidgets.VBox
        Controls + live Matplotlib output area.
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import (
        FloatSlider, IntSlider, Play, jslink, Checkbox, HBox, VBox, Layout, Output, Label, Button
    )
    from IPython.display import display, clear_output

    # -----------------------
    # Fixed simulation grid
    # -----------------------
    L = 20.0          # half-length of rope domain [-L, L]
    N = 800           # spatial samples
    N_FRAMES = 400    # animation frames
    DT = 0.02         # seconds per frame
    VELOCITY = 5.0    # wave speed (m/s)

    x = np.linspace(-L, L, N)
    x0 = 15.0              # initial offset of packets from center
    DEFAULT_SIGMA = 2.0    # packet width (std dev of Gaussian)

    # -----------------------
    # Widgets
    # -----------------------
    amplitude_L = FloatSlider(value=1.0, min=0.1, max=2.5, step=0.1,
                              description="A (left pkt)", readout_format=".2f")
    amplitude_L.style.description_width = 'initial'
    amplitude_R = FloatSlider(value=1.0, min=0.1, max=2.5, step=0.1,
                              description="A (right pkt)", readout_format=".2f")
    amplitude_R.style.description_width = 'initial' 
    wavelength = FloatSlider(value=4.0, min=1.0, max=10.0, step=0.5,
                             description="Wavelength λ", readout_format=".2f")
    wavelength.style.description_width = 'initial'
    width = FloatSlider(value=DEFAULT_SIGMA, min=0.5, max=5.0, step=0.1,
                        description="Packet width σ", readout_format=".1f")
    width.style.description_width = 'initial'
    show_components = Checkbox(value=True, description="Show individual waves")
    play = Play(value=0, min=0, max=N_FRAMES-1, step=1, interval=30)
    frame = IntSlider(value=0, min=0, max=N_FRAMES-1, step=1, description="Frame")
    jslink((play, 'value'), (frame, 'value'))

    reset_btn = Button(description="Reset", button_style='info')
    save_btn = Button(description="Save Figure", button_style='success')

    readout = Label()
    controls_top = HBox([amplitude_L, amplitude_R, wavelength],
                        layout=Layout(column_gap='12px'))
    controls_mid = HBox([width, show_components],
                        layout=Layout(column_gap='12px', align_items='center'))
    controls_bottom = HBox([play, frame, reset_btn, save_btn],
                           layout=Layout(column_gap='12px', align_items='center'))

    # -----------------------
    # Figure / Output
    # -----------------------
    out = Output()
    fig, ax = plt.subplots(figsize=(10, 4))
    plt.close(fig)  # prevent Jupyter/Voila from capturing a static image

    # -----------------------
    # Drawing routine
    # -----------------------
    def _draw(idx: int):
        A_L = float(amplitude_L.value)
        A_R = float(amplitude_R.value)
        lam = float(wavelength.value)
        v = VELOCITY
        sigma = float(width.value)

        k = 2 * np.pi / lam
        t = idx * DT

        # Centers moving toward the origin
        xL = -x0 + v * t   # left packet center moving right
        xR = +x0 - v * t   # right packet center moving left

        # Two Gaussian-windowed sinusoids (short "pulses")
        y1 = A_L * np.sin(k * (x - xL)) * np.exp(-((x - xL) ** 2) / (2 * sigma ** 2))
        y2 = A_R * np.sin(k * (x - xR)) * np.exp(-((x - xR) ** 2) / (2 * sigma ** 2))
        y = y1 + y2

        with out:
            ax.cla()

            # Baseline rope
            ax.axhline(0, lw=1, alpha=0.5)

            # Sum (what the rope actually does)
            ax.plot(x, y, lw=2, label="Sum (superposition)")

            # Optional components
            if show_components.value:
                ax.plot(x, y1, '--', lw=1.2, label="Packet from left →")
                ax.plot(x, y2, '--', lw=1.2, label="Packet from right ←")

            # Cosmetics
            ax.set_xlim(-L, L)
            # Scale limits to accommodate full constructive interference
            amp_sum = max(0.1, A_L + A_R)
            ax.set_ylim(-2.2 * amp_sum, 2.2 * amp_sum)
            ax.set_xlabel("Position along rope (m)")
            ax.set_ylabel("Amplitude (displacement in meters)")
            ax.set_title("Wave Superposition on a Rope (Two Opposing Wave Packets)")
            ax.grid(True, alpha=0.3)
            ax.legend(loc="upper right", frameon=False)

            # Readout text
            f = v / lam  # Hz
            readout.value = (
                f"A_left = {A_L:.2f},  A_right = {A_R:.2f},  λ = {lam:.2f} m,  "
                f"v = {v:.2f} m/s,  f = {f:.2f} Hz,  t = {t:.2f} s"
            )

            clear_output(wait=True)
            display(fig)

    # -----------------------
    # Handlers
    # -----------------------
    def _reset(_):
        amplitude_L.value = 1.0
        amplitude_R.value = 1.0
        wavelength.value = 4.0
        width.value = DEFAULT_SIGMA
        show_components.value = True
        frame.value = 0
        play.value = 0
        _draw(0)

    def _save(_):
        import os
        os.makedirs('outputs', exist_ok=True)
        A_L = float(amplitude_L.value)
        A_R = float(amplitude_R.value)
        lam = float(wavelength.value)
        sigma = float(width.value)
        fname = f"outputs/rope_superposition_AL{A_L:.1f}_AR{A_R:.1f}_lam{lam:.1f}_sigma{sigma:.1f}_frame{int(frame.value)}.png"
        fig.savefig(fname, dpi=150, bbox_inches='tight')
        print(f"Saved {fname}")

    reset_btn.on_click(_reset)
    save_btn.on_click(_save)

    # -----------------------
    # Wiring
    # -----------------------
    for w in (amplitude_L, amplitude_R, wavelength, width, show_components):
        w.observe(lambda _: _draw(frame.value), names='value')

    # Frame playback
    frame.observe(lambda ch: _draw(ch['new']), names='value')

    # Initial render
    _draw(frame.value)

    return VBox([controls_top, controls_mid, controls_bottom, readout, out])

rope_app = build_rope_superposition_sim()
display(rope_app)

_Figure: Two short wave packets travel toward each other and interfere; changing amplitudes, wavelength, or width alters the interference pattern._

### 🧪 Try this
- Set equal amplitudes for strong **constructive interference** at the meeting point.
- Make one packet larger to see **partial reinforcement** instead of full cancellation.
- Narrow or widen the packet width to change how long the overlap lasts.
- Adjust wavelength to compress or stretch ripples within each packet.


### **Wave Superposition in Action: Pulse Transmission in Radar** 📡⚡  

We’re just getting warmed up! Now we are ready to come back to the core idea we mentioned in the beginning of this notebook: **building a signal** by **adding together simple waves** with different parameters.

In **radar systems**, we often need to transmit **pulses** where the amplitude isn’t a smooth sine wave but instead has a **rectangular** or otherwise non-sinusoidal shape.

> _(We’ll explore this in more detail when we discuss **modulated chirp waveforms** later on. 🎶)_

At first glance, creating such a shape from sinusoids might seem impossible. However, thanks to the **principle of superposition**, we can approximate it remarkably well by summing the right combination of waves with **different frequencies and amplitudes**.

---

## **Wave Addition Experiment** 🧪  

The next simulation lets you see this in action. 🖥️  

- **Adjust the number of harmonic waves** to watch how the sum changes. 🔢  
- Each harmonic has a **specific amplitude** and **frequency**, but shares the **same phase**.  
- Observe how combining them reshapes the waveform — from a smooth sine toward a sharp-edged pulse. 📈
- Notice how the more harmonics you add, the closer you get to the desired rectangular pulse shape. 📦

In [ ]:
def build_fourier_superposition_sim_fixed_fs():
    """
    Voila-ready interactive simulation:
    Sum odd harmonics of a base sine to illustrate wave superposition (square-wave series).
    Sampling frequency is fixed internally.
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import FloatSlider, IntSlider, Checkbox, HBox, VBox, Layout, Output, Label, Button
    from IPython.display import display, clear_output

    # -----------------------
    # Colors (from your snippet)
    # -----------------------
    colors = {
        'green': '#71B17F',
        'purple': '#471D6F'
    }

    # -----------------------
    # Fixed parameters
    # -----------------------
    FS = 1000         # Fixed sampling frequency [Hz]
    DURATION = 2.0    # Fixed time window [s]

    # -----------------------
    # Widgets
    # -----------------------
    base_freq = FloatSlider(value=2.0, min=0.2, max=10.0, step=0.1,
                            description="Base f (Hz)", readout_format=".1f")
    base_freq.style.description_width = 'initial'
    n_harm = IntSlider(value=5, min=1, max=25, step=1,
                       description="# Harmonics")
    n_harm.style.description_width = 'initial'
    show_parts = Checkbox(value=True, description="Show components")
    clamp_ylim = Checkbox(value=True, description="Auto y-limits")
    reset_btn = Button(description="Reset", button_style='info')
    save_btn = Button(description="Save Figure", button_style='success')

    controls_row1 = HBox([base_freq, n_harm, show_parts],
                         layout=Layout(column_gap='12px', align_items='center'))
    controls_row2 = HBox([clamp_ylim, reset_btn, save_btn],
                         layout=Layout(column_gap='12px', align_items='center'))

    readout = Label()

    # -----------------------
    # Output / Figure
    # -----------------------
    out = Output()
    fig, ax = plt.subplots(figsize=(10, 6))
    plt.close(fig)  # prevent Voila from capturing static image

    # -----------------------
    # Draw routine
    # -----------------------
    def _draw():
        f0 = float(base_freq.value)
        Nh = int(n_harm.value)

        nsamps = int(DURATION * FS)
        t = np.linspace(0.0, DURATION, nsamps, endpoint=False)

        # Odd harmonics: 1,3,5,... (2k+1)
        odds = (2*np.arange(Nh) + 1).astype(float)

        # Vectorized computation of harmonics
        k = odds[None, :]                      # shape (1, Nh)
        phase = k * 2*np.pi*f0                 # angular frequency multiplier
        sines = np.sin(t[:, None] * phase)     # shape (nsamps, Nh)
        weights = (4.0 / np.pi) * (1.0 / k)    # shape (1, Nh)
        parts = sines * weights                # (nsamps, Nh)
        y_sum = np.sum(parts, axis=1)

        with out:
            ax.cla()

            # Optional: plot individual components
            if show_parts.value:
                label_cut = min(8, Nh)
                for i in range(Nh):
                    lbl = (f"{int(odds[i])}× harmonic" if i < label_cut else None)
                    ax.plot(t, parts[:, i],
                            color=colors['green'], alpha=0.6, lw=1.2,
                            label=lbl)
            # Composite signal
            ax.plot(t, y_sum, color=colors['purple'], lw=2.5, alpha=0.9,
                    label="Composite (sum)")

            ax.set_title('Making Arbitrary Waveforms by Summing Harmonic Signals')
            ax.set_xlabel('Time [s]')
            ax.set_ylabel('Amplitude')
            ax.grid(True, alpha=0.35)

            # Auto y-limits based on harmonic sum
            if clamp_ylim.value:
                theor = (4.0/np.pi) * np.sum(1.0/odds)
                pad = 0.15 * max(1.0, theor)
                ax.set_ylim(-(theor + pad), (theor + pad))
            else:
                ax.set_ylim(-6, 6)

            handles, labels = ax.get_legend_handles_labels()
            if labels:
                ax.legend(loc='upper right', frameon=False)

            readout.value = (
                f"f₀ = {f0:.2f} Hz  |  #odd harmonics = {Nh}  |  Fs = {FS} Hz  |  duration = {DURATION:.2f} s"
            )

            clear_output(wait=True)
            display(fig)

    # -----------------------
    # Handlers
    # -----------------------
    def _reset(_):
        base_freq.value = 2.0
        n_harm.value = 5
        show_parts.value = True
        clamp_ylim.value = True
        _draw()

    def _save(_):
        import os
        os.makedirs('outputs', exist_ok=True)
        f0 = float(base_freq.value)
        Nh = int(n_harm.value)
        fname = f"outputs/fourier_superposition_f0{f0:.1f}_Nh{Nh}.png"
        fig.savefig(fname, dpi=150, bbox_inches='tight')
        print(f"Saved {fname}")

    reset_btn.on_click(_reset)
    save_btn.on_click(_save)

    # -----------------------
    # Wiring
    # -----------------------
    for w in (base_freq, n_harm, show_parts, clamp_ylim):
        w.observe(lambda _ : _draw(), names='value')

    # Initial render
    _draw()

    return VBox([controls_row1, controls_row2, readout, out])

fourier_app_fixed = build_fourier_superposition_sim_fixed_fs()
display(fourier_app_fixed)

_Figure: Summing more odd harmonics sharpens edges and flattens plateaus, transforming a sine toward a pulse-like waveform._

### 🧪 Try this
- Increase the **number of harmonics** and watch edges sharpen.
- Change **base frequency** and see the waveform compress/expand in time.
- Hide components to focus on the composite; re-enable to inspect wave contributions.


## **Taking Samples of Our Precious Waves** 🌊  

Imagine sitting by a lake, watching gentle waves lap at your feet. The waves appear **continuous**—you might think that, with a precise enough instrument, you could measure their height **at any point**. However, this perception is misleading.  

Every **measurement takes time**, meaning we can never truly **capture a continuous physical signal**. ⏳  

### **The Challenge of Remembering a Wave** 🧠  

Now, imagine trying to recall the **exact shape** of the wave to describe this moment to a friend.  

- Once you leave the shore, it’s **impossible** to remember the wave’s height **at every location**. 💭  
- Your **brain, like any computer, has limited storage capacity**, so some information is inevitably lost. 💾  

### **Why Do We Need Sampling?**  

A **continuous signal** has a value at **every instant in time**. In theory, it would require **an infinite amount of memory** to store all it's values. Since this is impossible, we must convert the **continuous signal** into a **finite set of samples** to store and process it **digitally**. 🔢  

This process is called **sampling**. The best we can do is **sample the signal as frequently as possible**.  

> **This fundamental limitation is at the heart of all digital signal processing: every signal must be represented as a set of discrete measurements.**  

### **The Importance of Sampling Rate** 📡  

When sampling a **periodic signal**, such as the **oscillating electric field in a radar receiver**, we must be **careful**.  

- The way we choose our **sampling rate** (how rapidly we take samples from the signal) has a profound effect on the information we capture.  
- Sampling too slowly can lead to **distorted or misleading results**. ❌  
- If we sample too infrequently, we might not capture the true nature of the wave at all. 🔄  

## **Interactive Sampling Experiment** 🖥️  

Let's explore **how the sampling rate affects what we see in a wave signal**. By changing the **number of samples per wave cycle**, you can watch how the sampled points relate to the original continuous wave.

- **Increase** the samples per cycle to capture the wave more accurately — the sampled points will closely trace the original shape. 📈  
- **Reduce** the samples per cycle below **2** and see the **aliasing effect** in action — the reconstructed wave seems to change frequency! 🔄  


In [ ]:
def build_sampling_aliasing_sim_strict():
    """
    Voila-ready version of your sampling + phasor animation.
    Physics and calculations are kept IDENTICAL to the provided script.
    """
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches
    from ipywidgets import FloatSlider, Play, IntSlider, jslink, HBox, VBox, Layout, Output, Button
    from IPython.display import display, clear_output

    # -----------------------
    # Widget(s) matching your "## CHANGE THIS VALUE"
    # -----------------------
    samples_per_cycle_w = FloatSlider(value=3, min=0.2, max=6.0, step=0.1,
                                      description="Samples/cycle", readout_format=".2f")
    samples_per_cycle_w.style.description_width = 'initial'
    play = Play(value=0, min=0, max=10, step=1, interval=1000)  # interval = 1e3 ms (same as your script)
    frame = IntSlider(value=0, min=0, max=10, step=1, description="Frame")
    jslink((play, 'value'), (frame, 'value'))
    reset_btn = Button(description="Reset", button_style='info')
    save_btn = Button(description="Save Figure", button_style='success')

    controls = HBox([samples_per_cycle_w, play, frame, reset_btn, save_btn],
                    layout=Layout(align_items='center', column_gap='12px'))

    # -----------------------
    # Output / Figure
    # -----------------------
    out = Output()
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    plt.close(fig)  # prevent static capture in Voila

    # -----------------------
    # State computed EXACTLY like your script
    # -----------------------
    def recompute_state():
        # --- Your variables (kept identical) ---
        samples_per_cycle = float(samples_per_cycle_w.value)

        amplitude = 1.0
        frequency = 1.0
        initial_phase = np.pi / 2
        omega = 2 * np.pi * frequency
        sampling_frequency = samples_per_cycle * frequency

        # Aliasing branch & folding frequency (unchanged)
        folding_frequency = None
        if sampling_frequency < 2 * frequency:
            n = np.floor(np.abs(frequency) / (sampling_frequency / 2))
            folding_frequency = np.min([frequency - n * sampling_frequency,
                                        n * sampling_frequency - frequency])

        # Time arrays (unchanged)
        sampling_interval = 1 / (sampling_frequency)
        nb_cycles = 6
        total_time = nb_cycles / frequency
        nb_samples = np.round(total_time / sampling_interval)  # kept for parity
        time = np.arange(0, total_time, sampling_interval)

        # Prepare background lines (unchanged)
        time_full = np.linspace(0, total_time, 200)
        sinusoid_full = amplitude * np.sin(omega * time_full + initial_phase)

        folded_sinusoid = None
        omega_folded = None
        if sampling_frequency < 2 * frequency:
            omega_folded = 2 * np.pi * folding_frequency
            folded_sinusoid = amplitude * np.sin(omega_folded * time_full + initial_phase)

        return {
            'samples_per_cycle': samples_per_cycle,
            'amplitude': amplitude,
            'frequency': frequency,
            'initial_phase': initial_phase,
            'omega': omega,
            'sampling_frequency': sampling_frequency,
            'folding_frequency': folding_frequency,
            'sampling_interval': sampling_interval,
            'nb_cycles': nb_cycles,
            'total_time': total_time,
            'nb_samples': nb_samples,
            'time': time,
            'time_full': time_full,
            'sinusoid_full': sinusoid_full,
            'folded_sinusoid': folded_sinusoid,
            'omega_folded': omega_folded
        }

    state = recompute_state()

    # Sync frame range to number of samples (like frames = len(time))
    def sync_frame_limits():
        with frame.hold_sync(), play.hold_sync():
            max_idx = max(0, len(state['time']) - 1)
            frame.min = 0
            frame.max = max_idx
            frame.value = min(frame.value, max_idx)
            play.min = 0
            play.max = max_idx
            play.value = min(play.value, max_idx)
            # interval fixed to 1000 ms as in your original script
            play.interval = 1000

    sync_frame_limits()

    # -----------------------
    # Draw routine (faithful to your update/init)
    # -----------------------
    def _draw(idx: int):
        A = state['amplitude']
        omega = state['omega']
        initial_phase = state['initial_phase']
        time = state['time']
        time_full = state['time_full']
        sinusoid_full = state['sinusoid_full']
        folded_sinusoid = state['folded_sinusoid']
        sampling_frequency = state['sampling_frequency']
        frequency = state['frequency']

        # Compute per-frame values (unchanged)
        current_time = time[idx % len(time)]
        angle = omega * current_time

        real = (A - 0.1) * np.cos(angle + initial_phase)
        imag = (A - 0.1) * np.sin(angle + initial_phase)
        x_value = current_time
        y_value = A * np.sin(angle + initial_phase)

        with out:
            ax1.cla()
            ax2.cla()

            # --- Phasor subplot (unchanged style) ---
            ax1.set_aspect('equal')
            ax1.set_xlim(-A * 1.2, A * 1.2)
            ax1.set_ylim(-A * 1.2, A * 1.2)
            ax1.set_title("Phasor Representation")
            ax1.set_xlabel("I (real part)")
            ax1.set_ylabel("Q (imaginary part)")
            ax1.grid(True)
            arrow = patches.FancyArrow(0, 0, real, imag, head_width=0.05, head_length=0.1, color='blue')
            ax1.add_patch(arrow)

            # --- Time-domain subplot (unchanged logic) ---
            ax2.set_xlabel("Time [s]")
            ax2.set_ylabel("Amplitude")
            ax2.grid(True)

            # Red dashed true sine
            ax2.plot(time_full, sinusoid_full, 'r--', lw=1, label="Sine Wave")

            # Blue dashed aliased sine if under Nyquist (unchanged condition)
            if sampling_frequency < 2 * frequency and folded_sinusoid is not None:
                ax2.plot(time_full, folded_sinusoid, 'b--', lw=1, label="Aliased Sine Wave")
                ax2.legend(loc='best')

            # Blue dot (projection) — unchanged
            ax2.plot([x_value], [y_value], 'bo', label="Signal Sample")
            ax2.legend(loc='best')

            clear_output(wait=True)
            display(fig)

    # -----------------------
    # Handlers
    # -----------------------
    def _reset(_):
        samples_per_cycle_w.value = 3.0
        frame.value = 0
        play.value = 0
        # Recompute and redraw from start
        nonlocal state
        state = recompute_state()
        sync_frame_limits()
        _draw(0)

    def _save(_):
        import os
        os.makedirs('outputs', exist_ok=True)
        spc = float(samples_per_cycle_w.value)
        fname = f"outputs/sampling_aliasing_spc{spc:.2f}_frame{int(frame.value)}.png"
        fig.savefig(fname, dpi=150, bbox_inches='tight')
        print(f"Saved {fname}")

    reset_btn.on_click(_reset)
    save_btn.on_click(_save)

    # -----------------------
    # Wiring
    # -----------------------
    def on_param_change(_):
        nonlocal state
        state = recompute_state()
        sync_frame_limits()
        _draw(frame.value)

    samples_per_cycle_w.observe(on_param_change, names='value')
    frame.observe(lambda ch: _draw(ch['new']), names='value')

    # Initial render
    _draw(frame.value)

    return VBox([controls, out])

sampling_app = build_sampling_aliasing_sim_strict()
display(sampling_app)

_Figure: Changing the sampling density alters how the blue samples trace the red sine; below 2 samples per cycle, they follow an aliased (apparent) slow sinusoid._

### 🧪 Try this
- Increase to 3 samples/cycle: the dots closely trace the true sine.
- Set below 2 samples/cycle: observe the slower apparent oscillation (aliasing).


## **Aliasing: When Signals Deceive Us** 🎭  

When a **phasor** completes **half a revolution**, it corresponds to **half a cycle** of a sinusoidal wave—equivalent to **half a wavelength** of the signal.  

But what happens if we take **fewer than two samples per cycle**?  

- The wave appears to behave **differently than it actually does**. 🔄  
- In fact, it can **look as if the phasor is rotating in a different direction**! ❌  
- This misleading effect is called **aliasing**, where a signal is incorrectly interpreted as having a **different frequency** than its true value. 🎭  

### **The Sampling Theorem and Nyquist Rate** 📡  

This brings us to a fundamental principle in **signal processing**:  

> **The sampling theorem (Nyquist theorem)**—named after Swedish-American engineer **Harry Nyquist**.  

The theorem states that to **accurately reconstruct a signal**, we must sample it **at least twice per wave cycle**.  

In engineering terms, this means:  
- The **sampling rate** must be at least **twice the highest frequency** present in the signal. 📈  
- This critical rate is called the **Nyquist rate**.  

If the sampling rate is **lower than the Nyquist rate**, aliasing occurs, causing **misinterpretation** of the signal's frequency. ❌  

---

## **Sampling Theorem Applied to Radar** 📡  

In radar systems, the phase of the received signal depends on the **distance the wave has traveled**—specifically, the **range between the radar and the reflecting target**. This has crucial implications for processing radar signals over time. ⏲️  

Many radar systems, including **Synthetic Aperture Radar (SAR)**, repeatedly measure the **same target over time**. Between these measurements, the **distance between the radar and the target changes**.  

For SAR satellites, the radar moves relative to the ground surface, meaning that echoes from the same ground location will have **different phases** at different measurement points. This variation occurs because the **satellite travels with its orbital velocity**, shifting its position from one measurement to the next. 🛰️  

Now, let’s apply the **sampling theorem** to this problem.  To **accurately reconstruct the signal** from one measurement to the next, the **change in phase between two measurements must not exceed half a cycle**.  

- A **phase shift of half a cycle** corresponds to a **distance of half a wavelength**. 📏  
- For our **SAR system**, this distance is **1.5 cm** (since the wavelength is 3 cm).  
- This means that the **range (distance) between the radar and the ground must not change by more than 1.5 cm** between two received pulses—otherwise, we can run into problems. ❌  
  
So what happens if this limit is broken? We’ll uncover the full story soon, but here’s a hint: it makes *phantom echoes* appear in the image—ghostly target echoes that are misplaced from their true locations. 👻


## 📌 Summary

- **Phasors** provide a compact way to link **amplitude** and **phase** to a **sine wave**.  
- **Adding waves** explains **interference patterns** and shows how **complex waveforms** can be built from simple ones.  
- **Sampling below the Nyquist rate** produces **aliasing**, where the **apparent frequency** of the signal differs from the **true frequency**.  
